# Data Exploration & Process to define Primary Key

In this Notebook, I am exploring the data and writing my thoughts process to support the decisions taken to choose the primary key for each question. 

Note: It has to be noted that decisions were taken for the specific subset of data used in this project and might not be valid if we took all the data. Since no information was given on the data collected, I used analytics to define how the data are associated.

In [1]:
import pandas as pd
import csv

In [2]:
# Look at the data: columns, types, etc.

df = pd.read_csv('./event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


## Question 1 :

**Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

In [3]:
print("--> The user named 'Ava Robison' has {} unique session_id\n".format(len(df.loc[(df['lastName']=='Robinson') & (df['firstName']=='Ava')]['sessionId'].unique())))
print("--> The user_id '50' corresponding to 'Ava Robinson' living in CT has {} unique session_id\n".format(len(df.loc[df['userId']==50]['sessionId'].unique())))
print("--> The user_id '50' corresponding to 'Ava Robinson' living in London has {} unique session_id\n".format(len(df.loc[df['userId']==13]['sessionId'].unique())))

df.loc[df['sessionId']==338]

--> The user named 'Ava Robison' has 26 unique session_id

--> The user_id '50' corresponding to 'Ava Robinson' living in CT has 23 unique session_id

--> The user_id '50' corresponding to 'Ava Robinson' living in London has 3 unique session_id



,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


**Users Information**
1. The same user can have multiple session_id
2. Two users (user_id) can have the same name (first+Last)
3. While the two users named 'Ava Robinson' don't live in the same city, there is a possibility that two users with the same name live in the same city

**To verify next:**

I want to confirm if the sessionId is unique to a user or unique to the database:
- *Unique by user* : each user has a unique set of sessionId, AND multiple users can have the same sessionId.
- *Unique to the database* : each new session started, for any given user, get its own new & unique sessionId. Meaning that there is no duplicate sessionId for different user

This information is most important because if two users can have the same sessionId: userId will have to be added in the primary key to ensure that the primary key is unique. Otherwise, data from previous users will be replaced by the latest, and we will data will be lost. 

In [4]:
df = pd.read_csv('./event_datafile_new.csv') # Re-Load data in case changes were made my mistake in previous cells

# How many unique userId per sessionId (can multiple user have the same session ID)
multi_user = df.groupby(['sessionId', 'userId']).count().reset_index()[['sessionId', 'userId']]
multi_user = multi_user.groupby('sessionId').count()

if len(multi_user.loc[multi_user['userId']>1]) > 0:
    print('--> Conclusion : At least one sessionId has more than one userId\n')

else:
    print('--> Conclusion : There one sessionId is specific to one userId\n')

print('Show all sessionId that has more than 1 userId:')
multi_user.loc[multi_user['userId']>1]

--> Conclusion : There one sessionId is specific to one userId

Show all sessionId that has more than 1 userId:


,userId
sessionId,


**Primary Key** 
1. Session_id is linked to a specific user (first name, gender, last name, level), making it perfect for a Partition Key. 
2. Each session_id is composed of rows about the songs listen to by the user (artist, song name, duration)
3. The incremental variable itemInSession uniquely identifies each song in a session. 


### Question 1 : Primary Key 

- Partition Key should be : sessionId
- Clustering Key should be : itemInSession

# Question 2:

**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

- Notes: As seen previously, the sessionId always corresponds to a unique userId; making the use of userId obsolete in this query. If only the three queries from this project were used, it would be an option to use the table from the previous query (with the needed variables to display added to the table) to reduce data storage and use the following query: (""" SELECT artist, song, firstName, lastName FROM sessions_library WHERE sessionId = 10 """). 

## Tought Process

1. In this query, we first want to look at a specific user. Since we have many users and each has a specific userId, UserId is an excellent choice to partition our data. 
2. Then, since we want to look at a specific sessionId for this specific user, we can add a second partition key: sessionId. SessionId is chosen as a partition key (and not a clustering) because each session contains more information about the songs played.
3. As noted in the previous query, to make a primary key unique when using the sessionId: we need to add the itemInSession to obtain all the songs played in this session (otherwise, we will only have one song per session)
4. The clustering key 'itemInSession' also allows to rank by this variable which is requested

### Question 2 : Primary Key 
- Partition Key should be : userId, sessionId
- Clustering Key should be : itemInSession

In [5]:
# Answer is suppose to have the following rows...

df = pd.read_csv('./event_datafile_new.csv') # Re-Load data in case changes were made my mistake in previous cells
df.loc[(df['sessionId']==182) & (df['userId']==10)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


# Question 3:

**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

## Thought Process

1. Since we want to find a specific song, we should partition the data by the song title. 
2. It is possible that multiple artists have the same song title, but no information was given. Thus, we assume that any given artist that played this song title should be displayed by the query even if the user listened to the same title from different artists. 
3. We already know from the first query that two people can have the same name, so we should add the userId as a clustering key to ensure that we can get each user that listened to the song (even if they have the same name!) --  versus using the name directly. 
4. Since we don't really care when they listen to the song (sessionId) or if they listen to it multiple time, we don't have to add the sessionId and itemInSession as clustering key. 

### Question 3 : Primary Key 
- Partition Key should be : song_title
- Clustering Key should be : userId

In [6]:
# Answer is suppose to have the following rows...

df = pd.read_csv('./event_datafile_new.csv') # Re-Load data in case changes were made my mistake in previous cells
df.loc[df['song']=='All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
